In [4]:
from transformers import pipeline
import torch

In [8]:
gen = pipeline("token-classification", "Isotonic/distilbert_finetuned_ai4privacy", device=-1, torch_dtype=torch.float32)

In [9]:
text = "My name is Claraly and I live in Berkeley, California."

In [20]:
output = gen(text, aggregation_strategy="simple")

In [21]:
output

[{'entity_group': 'USERAGENT',
  'score': 0.8568921,
  'word': 'responsive 6. 0 +',
  'start': 50,
  'end': 65},
 {'entity_group': 'USERAGENT',
  'score': 0.8719265,
  'word': 'retina 4. 0 +',
  'start': 67,
  'end': 78},
 {'entity_group': 'USERAGENT',
  'score': 0.7608481,
  'word': 'para',
  'start': 80,
  'end': 84},
 {'entity_group': 'USERAGENT',
  'score': 0.9923622,
  'word': '##x 3. 0 + turbo 2. 0 +',
  'start': 87,
  'end': 104},
 {'entity_group': 'USERAGENT',
  'score': 0.888705,
  'word': 'mobilia 5. 0 +',
  'start': 106,
  'end': 118}]

In [22]:
def replace_entities(output, text):
    word_to_entity_group = dict(
    (text[token["start"] : token["end"]], token["entity_group"]) for token in output
)
    for i, token in enumerate(output):
        word = list(word_to_entity_group.keys())[i]
        text = text.replace(word, f"[{word_to_entity_group[word]}]")

    return text

In [23]:
text = """These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using? On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings! Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail. Does this feature apply to all sections of the theme or just specific ones as listed in the text material?"""

In [24]:
replace_entities(output, text)

"These instructions apply to section-based themes ([USERAGENT], [USERAGENT], [USERAGENT]lla[USERAGENT], [USERAGENT]). What theme version am I using? On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings! Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail. Does this feature apply to all sections of the theme or just specific ones as listed in the text material?"

In [55]:
from transformers import pipeline

In [57]:
from transformers import Pipeline

transformers.pipelines.base.Pipeline